### JS's Promise.resolve equivalent

In JS, you can wrap a value in a dummy promise, if the value is itself a promise, returns itself

In [9]:
asyncio.run(resolve(10))

10

In [10]:
import asyncio
from collections.abc import Awaitable


async def resolve(coro):
    if isinstance(coro, Awaitable):
        return await coro
    return coro

In [11]:
asyncio.run(resolve(10))

10

In [12]:
async def foo():
    return 42

asyncio.run(resolve(foo()))

42

## @sync decorator

A decorator that turns a async function into a normal function, auto calling asyncio.run

In [1]:
import asyncio
from functools import wraps


def sync(f):
    @wraps(f)
    def _sync(*args, **kwargs):
        return asyncio.run(f(*args, **kwargs))
    return _sync

In [2]:
@sync
async def foo():
    return 42

foo()

42

## @agnostic decorator

Turns a normal coroutine function into a agnostic version of it, that is, if called outside a running loop, awaits it with asyncio.run

In [11]:
import asyncio
from functools import wraps
from collections.abc import Awaitable


def agnostic(f):
    @wraps(f)
    def _agnostic(*args, **kwargs):
        result = f(*args, **kwargs)
        if not isinstance(result, Awaitable):
            return result
        try:
            asyncio.get_running_loop()
            return result
        except RuntimeError:
            return asyncio.run(result)
    return _agnostic

In [14]:
@agnostic
async def foo():
    return 42

foo()

42

In [15]:
async def main():
    return await foo()

asyncio.run(main())

42

In [16]:
@agnostic
def sync_foo():
    return 42

sync_foo()

42